# Extract named entities from text

Identify and extract people, organizations, locations, dates, and other entities from text using LLMs.


## Problem

You have unstructured text containing important information—names, companies, dates, locations—that you need to extract and structure for analysis, search, or integration with other systems.

| Source | Extract |
|--------|---------|
| News articles | People, organizations, locations mentioned |
| Customer feedback | Product names, feature requests |
| Legal documents | Parties, dates, monetary amounts |


## Solution

**What's in this recipe:**
- Extract entities as structured JSON
- Use OpenAI's structured output for reliable parsing
- Access extracted entities as queryable columns

You use structured output to get entities in a consistent JSON format. The entities are stored as JSON columns that you can query and filter.


### Setup


In [ ]:
%pip install -qU pixeltable openai


In [ ]:
import os
import getpass

if 'OPENAI_API_KEY' not in os.environ:
    os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key: ')


In [ ]:
import pixeltable as pxt
from pixeltable.functions.openai import chat_completions
import json


In [ ]:
# Create a fresh directory
pxt.drop_dir('entities_demo', force=True)
pxt.create_dir('entities_demo')


### Define entity extraction schema


In [ ]:
# Define the JSON schema for entity extraction
entity_schema = {
    "type": "json_schema",
    "json_schema": {
        "name": "entities",
        "strict": True,
        "schema": {
            "type": "object",
            "properties": {
                "people": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": "Names of people mentioned"
                },
                "organizations": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": "Names of companies, institutions, or groups"
                },
                "locations": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": "Geographic locations (cities, countries, addresses)"
                },
                "dates": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": "Dates or time references"
                }
            },
            "required": ["people", "organizations", "locations", "dates"],
            "additionalProperties": False
        }
    }
}


### Create extraction pipeline


In [ ]:
# Create table for articles
articles = pxt.create_table(
    'entities_demo.articles',
    {'title': pxt.String, 'content': pxt.String}
)


In [ ]:
# Add entity extraction column
extraction_prompt = 'Extract all named entities from the following text:\n\n' + articles.content

articles.add_computed_column(
    extraction_response=chat_completions(
        messages=[{'role': 'user', 'content': extraction_prompt}],
        model='gpt-4o-mini',
        response_format=entity_schema
    )
)


In [ ]:
# Extract the entities JSON
articles.add_computed_column(
    entities=articles.extraction_response.choices[0].message.content
)


### Extract entities from text


In [ ]:
# Insert sample articles
sample_articles = [
    {
        'title': 'Tech Acquisition',
        'content': 'Microsoft announced today that CEO Satya Nadella will lead the acquisition of a Seattle-based startup. The deal, expected to close in March 2024, is valued at $500 million.'
    },
    {
        'title': 'Sports Update',
        'content': 'LeBron James led the Los Angeles Lakers to victory against the Boston Celtics on Tuesday night at Staples Center. Coach Darvin Ham praised the teams performance.'
    },
    {
        'title': 'Research Breakthrough',
        'content': 'Dr. Sarah Chen at Stanford University published groundbreaking research on renewable energy. The study, funded by the National Science Foundation, was conducted in Palo Alto, California.'
    },
]

articles.insert(sample_articles)


In [ ]:
# View extracted entities
for row in articles.select(articles.title, articles.entities).collect():
    print(f"\n{row['title']}:")
    entities = json.loads(row['entities'])
    for entity_type, values in entities.items():
        if values:
            print(f"  {entity_type}: {', '.join(values)}")


## Explanation

**Structured output ensures reliable extraction:**

By using OpenAI's structured output (`response_format`), the model always returns valid JSON matching the schema. No post-processing or error handling needed.

**Common entity types:**

| Entity Type | Examples |
|-------------|----------|
| People | Names, titles |
| Organizations | Companies, institutions |
| Locations | Cities, countries, addresses |
| Dates | Specific dates, time periods |
| Money | Amounts, currencies |
| Products | Brand names, model numbers |

**Customizing the schema:**

Modify the `entity_schema` to extract domain-specific entities—product SKUs, legal terms, medical conditions, etc.


## See also

- [Extract structured data from images](https://docs.pixeltable.com/howto/cookbooks/images/vision-structured-output) - JSON extraction from images
- [Extract fields from JSON](https://docs.pixeltable.com/howto/cookbooks/core/workflow-json-extraction) - Parse LLM response fields
